# 07_Correlation Maps

In this chapter, we will learn how to produce correlation plots in 1D (scatter plots) and 2D (maps). As an example we will take the prominent feature of the North Atlantic Oscillation (NAO).

Some word about this phenomena:

Over the middle and high latitudes of the Northern Hemisphere, especially during the cold season months(November-April), the most prominent and recurrent pattern of atmospheric variability is the NAO (James Hurrel et al., 2003).  It is a weather phenomenon of fluctuations in the difference of atmospheric pressure at sea level (SLP) between the Icelandic Low and the Azores High. Through fluctuations in the strength of the Icelandic low and the Azores high, it controls the strength and direction of westerly winds and location of storm tracks across the North Atlantic (James Hurrel et al., 2003). It is a perfect example for correlation: in phases of positive NAO+, both the Azores High as well as the Icelandic Low are well pronounced compared to the mean over time. In contrast for phases of NAO-, both are rather weak.

See the plots below from Wanner, H., Brönnimann, S., Casty, C., et al. (2001):

**NAO+**

<img src="https://www.dropbox.com/s/1amtd1c5kwuzgtm/wanner_etal_01_naoplus.jpg?dl=1" width="80%"  align="left">

**NAO-**

<img src="https://www.dropbox.com/s/tiassrtblrryfma/wanner_etal_01_naominus.jpg?dl=1" width="80%"  align="left">

In [ ]:
# Display the plots in the notebook:
%matplotlib inline

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
import pandas as pd  # new package! this is the package at the base of xarray
#from eofs.xarray import Eof  # new package! http://ajdawson.github.io/eofs/index.html
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore')


## 07_01 One Point Correlation Maps

A one point correlation map correlates a variable all over the globe with the same variable at a specific site.

Our aim is now, to correlate the northern hemisphere winter season geopotential @500 hPa to the winter season geopotential on Iceland. We expect a highly negative correlation between Iceland and the Azores! We will first need to load the data:

In [ ]:
geop = xr.open_dataset('./data/ERA5-LowRes-MonthlyAvg-500hPa-UVZ.nc').sel(latitude=slice(90, 20)).z

We loaded the dataset, selected the northern hemisphere and the variable z all in on step. See what we got:

In [ ]:
geop = xr.open_dataset('./data/ERA5-LowRes-MonthlyAvg-500hPa-UVZ.nc').sel(latitude=slice(90, 20)).z
geop

Divide this by $9.8$ to get the geopotential height out of the geopotential!

In [ ]:
geop = geop/9.8

That worked out well. Next, we need to get the two variables we need for the correlation: we need to select the winter season DJF plus take an average over each season and we need to create another variable, the geopotential of iceland (select lon and lat).

In [ ]:
# create the northern hemisphere variable
geop_djf = geop.where(geop['time.season'] == 'DJF') #select winter season since correlation is strongest here
geop_djf = geop_djf.groupby('time.year').mean('time') #average over the year (over each season)

In [ ]:
# create the iceland variable
iceland = geop_djf.sel(latitude=65, longitude=-30, method='nearest') #select the lat & lon coords of iceland
iceland

What we aim to compute next is the correlation coefficient $r_{X,Y}$ of those two variables: 

$r_{X,Y} = \frac{cov(X,Y)}{\sigma_X\sigma_Y}$


$cov(X,Y) = \overline{(X-\overline{X})(Y-\overline{Y})}$

$r_{X,Y}$ can take values between $-1$ and $+1$, where $-1$ means perfect negative correlation and $+1$ perfect positive correlation of the two variables $X$ and $Y$.

To compute $r_{X,Y}$ of two specific sites is quite easy, since only the time dimension remains and the other coordinates (lat and lon) vanish. There is a special numpy method, that does exactly this: `np.corrcoef(X,Y)`.
Let's try this first:

In [ ]:
azores = geop_djf.sel(latitude = 38, longitude = -25.5, method = 'nearest') #select lat,lon of azores
r_xy = np.corrcoef(iceland,azores)
r_xy

The numpy function returns a matrix of the following form:

$\left(\begin{array}{rr}                                
r_{X,X}  &  r_{X,Y}\\                                               
r_{Y,X}  &  r_{Y,Y} \\                                                                                            
\end{array}\right)$

so what we need is the upper right entry of this matrix. We can select it with `[0, 1]`:

In [ ]:
r_xy [0,1] #select first row, second column

We can also plot a scatter plot of the geopotential of the azores vs. iceland with `plt.scatter(X,Y)`. We can clearly see the negative correlation in there too:

In [ ]:
plt.scatter(azores, iceland)
plt.xlabel('Geopotential Height Azores [m]');
plt.ylabel('Geopotential Height Iceland [m]');
plt.title('Negative Correlation of Geopotential Height');

From this, we can already conclude that the icelandic low and the azores high are correlated. Still, a map would be more appealing to see which regions correlate and which do not. Therefore, we must compute the correlation coefficient for each grid point on the northern hemisphere. We can do this by two so called **for loops**!

What is a for loop?

A loop is a tool in programming, to repeat the same command for different variables with only one time writing the command itself. A for loop iterates over those variables. Let's try it on an example:


In [ ]:
for i in list([1,2,3,4,5]): 
    print(i)

The for loop picks the i'th value out of the list, starting at $i=0$. Next, the command inside the for loop is executed: the ith list value (in this case 1) is printed. Next the for loop iteration selects $i=1$ and the execution of the print command prints 2, and so on until the end of the list is reached.
We can now also generate a nested set of two for loops:


In [ ]:
for i in list([1,2,3,4,5]):
    for j in list([6,7,8,9,10]):
        print(i,j)
        

We start with $i=0$, so $1$ is printed. The $i$ value stays constant, while we loop through all the $j$ values! Once the j-loop is completed, $i$ is set to $1$ (the second value of the list is printed: $2$).

That's exactly what we need for our correlation coefficient map, that persists of the correlation coefficient at each grid point! We simply need to iterate over all latitudes and longitudes in a nested for loop and compute the correlation coeffiecient with `np.corrcoef(X,Y)` for each grid point. See the explanations underneath each line:

In [ ]:
# first, we make an empty array that we will fill with the correlation coefficients of each grid point
cor_map = geop_djf[0,:,:] * 0.
# for loops over lats and lons
for j in np.arange(len(geop_djf.latitude)):
#len(geop_djf.latitude) returns the lengts of the latitude coordinate. np.arange(-"-) therefore creates an array 
#starting from 0 to len(geop_djf.latitude) --> those are the indexes we iterate with!
    for i in np.arange(len(geop_djf.longitude)):
        cor_map.values[j, i] = np.corrcoef(geop_djf.values[:, j, i], iceland.values)[0, 1]
        # for every lat, lon combination (j,i) we compute the np.corrcoef and safe it to the respective j,i matrix entry
        # of the predefined cor_map
        # geop_djf.values[:,j,i] selects all values in time, but only the latitude and longitude that we are dealing
        # with in this iteration (j,i)
        # iceland only has one lat and lon, so nothing needs to be selected
        # we use the .values attribute because this is much faster

Let's have a look if the just created `cor_map` looks as expected:


In [ ]:
cor_map

Now, we can plot the result of the correlation: the correlation coefficient! To get a nicer view on the northern hemisphere, we select a different projection, the North Polar Stereo Projection. 

In [ ]:
fig = plt.figure(figsize=(9, 7))
ax = plt.axes(projection=ccrs.NorthPolarStereo()) 
ax.set_extent([-180, 180, 20, 90], ccrs.PlateCarree())
ax.coastlines(); ax.gridlines();
cs = cor_map.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'label':'Correlation Coefficient'}, 
                           levels=np.linspace(-0.8, 0.8, 9), extend='both')
plt.title('One point correlation map (ref 65°N, 30°W)');

If you want, you can also make a circular plot instead of the quadratic one. First, you'll have to run these few lines (only once for the notebook!):

In [ ]:
import matplotlib.path as mpath
theta = np.linspace(0, 2*np.pi, 100)
map_circle = mpath.Path(np.vstack([np.sin(theta), np.cos(theta)]).T * 0.5 + [0.5, 0.5])

And then add one line to the plot commands:

In [ ]:
fig = plt.figure(figsize=(9, 7))
ax = plt.axes(projection=ccrs.NorthPolarStereo()) 
ax.set_boundary(map_circle, transform=ax.transAxes) #this is the only new line, the rest did not change
ax.set_extent([-180, 180, 20, 90], ccrs.PlateCarree())
ax.coastlines(); ax.gridlines();
cs = cor_map.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'label':'Correlation Coefficient'}, 
                           levels=np.linspace(-0.8, 0.8, 9), extend='both')
plt.title('One point correlation map (ref 65°N, 30°W)');

## 07_02 Correlation Maps

Another example of correlation maps would be to correlate to variables all over a certain area instead of one variable all over the area with the same variable at a certain location (one point correlationn maps).

We try this with the NAO index and the temperatures over europe in the winter season. Therefore, we must first get the NAO index. ython makes it very easy to read data directly from a given url:

In [ ]:
# import the modules we need
import io, requests

In [ ]:
# This just reads the data from an url
# data from http://www.cpc.ncep.noaa.gov/data/teledoc/nao.shtml
url = 'http://ftp.cpc.ncep.noaa.gov/wd52dg/data/indices/nao_index.tim'
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')), delim_whitespace=True, skiprows=7)
# Parse the time and convert to xarray
time = pd.to_datetime(df.YEAR.astype(str) + '-' + df.MONTH.astype(str))
nao = xr.DataArray(df.INDEX, dims='time', coords={'time':time})
# Select the ERA period
nao = nao.sel(time=slice('1979', '2014'))

See what we got:


In [ ]:
nao

In [ ]:
nao.plot()
plt.axhline(0, color = 'k');
plt.ylabel('NAO Index');
plt.xlim(['1979-01-01', '2014-12-01']);

To be able to create a scatter plot, we can now have a look the NAO index versus the temperatures over norway!
Load the temperature dataset, select and average over a part norway and take the winter season again:

In [ ]:
t2m = xr.open_dataset('./data/ERA5--LowRes-MonthlyAvg-t2m_tp.nc').t2m.sel(time = slice('1979','2014'))
t2m_nor = t2m.sel(latitude = slice(63,60), longitude = slice(8.5,10.5)).mean(dim = ['latitude', 'longitude']) - 273.15
t2m_nor

In [ ]:
t2m_nor = t2m_nor.where(t2m_nor['time.season'] == 'DJF') #select winter season since correlation is strongest here
t2m_nor = t2m_nor.groupby('time.year').mean('time') #average over the year (over each season)

nao = nao.where(nao['time.season'] == 'DJF') 
nao = nao.groupby('time.year').mean('time') 

Let's have a look at the scatter plot to get a first idea of the correlation:

In [ ]:
plt.scatter(nao, t2m_nor)
plt.xlabel('NAO Index');
plt.ylabel('Temperature over Norway [°C]');
plt.title('Positive Correlation')
plt.xlim([-1.2,1.0]);

Now, we want to create a map of europe with the correlation values of NAO Index with temperatures.
In order to do that, we need a for loop again. 

In [ ]:
t2m = t2m.where(t2m['time.season'] == 'DJF') #select winter season since correlation is strongest here
t2m = t2m.groupby('time.year').mean('time') #average over the year (over each season)

In [ ]:
# first, we make an empty array that we will fill with the correlation coefficients of each grid point
cor_map = t2m[0,:,:] * 0.
# for loops over lats and lons
for j in np.arange(len(t2m.latitude)):
    for i in np.arange(len(t2m.longitude)):
        cor_map.values[j, i] = np.corrcoef(t2m.values[:, j, i], nao.values)[0, 1]

In [ ]:
fig = plt.figure(figsize=(9, 7))
ax = plt.axes(projection=ccrs.EuroPP()) # take the projection for europe 
ax.coastlines(); ax.gridlines();
cs = cor_map.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs={'label':'Correlation Coefficient'}, 
                           levels=np.linspace(-0.8, 0.8, 9), extend='both')
plt.title('Correlation map DJF NAO Index and Temperature');

As already expected when trying to understand the plots from Wanner, H., Brönnimann, S., Casty, C., et al. (2001) at the very beginning of this chapter, NAO and temperatures over northern europe are highly positive correlated. This means, NAO+ causes the temperatures over northern europe to be higher than the mean.